In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.distributions.normal import Normal
import torch.optim as optim
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.nn import Parameter
from torch import nn
from torch.utils.data import Dataset, DataLoader

import scipy.stats
import time
import pickle
import os
import sys

In [2]:
!rm -r h-vae/

In [3]:
!git clone https://github.com/Daniil-Selikhanovych/h-vae

Cloning into 'h-vae'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 106 (delta 32), reused 97 (delta 26), pack-reused 0
Receiving objects: 100% (106/106), 400.62 KiB | 23.57 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [4]:
!pip install pyro-ppl

from pyro.nn import AutoRegressiveNN

     |████████████████████████████████| 614kB 12.6MB/s 


In [5]:
path_to_gaussians = os.path.join(os.getcwd(), 'h-vae', 'gaussians')
path_to_gaussians_api = os.path.join(path_to_gaussians, 'api')

sys.path.append(path_to_gaussians)
sys.path.append(path_to_gaussians_api)

In [6]:
from models import HVAE, NF, VB, IAF

In [7]:
class gaussian_dataset(Dataset):
    def __init__(self, x):
          'Initialization'
          self.x = x

    def __len__(self):
          'Denotes the total number of samples'
          return self.x.shape[0]

    def __getitem__(self, index):
          'Generates one sample of data'
          x_index = self.x[index]

          return x_index

In [7]:
def run_test(d, params, 
             batch_size=256,
             clip_value=None,
             dtype=torch.float32, 
             device=torch.device("cpu")):
    """ 
    Run the gaussian test with dimension d 
    """

    ######### Problem Specification

    # Data generation parameters
    prior_mu_z = torch.zeros(d, dtype=dtype)    # Prior mean
    prior_sigma_z = torch.eye(d, dtype=dtype)   # Prior covariance matrix

    # True model parameters
    num_range = torch.arange(-(d-1)/2, (d+1)/2, dtype=dtype)

    t_delta =  num_range / 5 

    print(f"true delta = {t_delta.numpy()}")

    if d == 1:
        t_sigma = torch.ones(1, dtype=dtype)
    else: 
        # Allow sigma to range from 0.1 to 1
        t_sigma = 36/(10*(d-1)**2) * num_range**2 + 0.1 
    print(f"true sigma = {t_sigma.numpy()}")

    ######### Variable Initialization

    # Initial model parameters - same across all methods
    init_delta = prior_mu_z.clone().to(device)
    init_log_sigma = 3 * torch.ones(d).to(device)

    # Initial HVAE variational parameters
    init_T = torch.tensor(5., dtype=dtype, device=device)
    init_eps = 0.005 * torch.ones(d, dtype=dtype)
    max_eps = params['max_eps'] * torch.ones(d, dtype=dtype)
    init_logit_eps = torch.log(init_eps/(max_eps - init_eps)).to(device)
    init_log_T_0 = torch.log(init_T - 1)

    # Initial NF variational parameters
    init_u_pre_reparam = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, 
                                                                scale=0.1, 
                                                                size=d),
                                      dtype=dtype, device=device)
    init_w = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, scale=0.1, size=d),
                          dtype=dtype, device=device)
    init_b = torch.tensor(0.1, dtype=dtype, device=device)

    # Initial VAE parameters
    init_mu_z = prior_mu_z.clone()
    init_log_sigma_z = torch.ones(d, dtype=dtype)

    ######### Set up models
    init_params = [[init_delta, init_log_sigma, init_logit_eps, init_log_T_0],
                   [init_delta, init_log_sigma, init_logit_eps, init_log_T_0],
                   [init_delta, init_log_sigma, init_u_pre_reparam, init_w, init_b],
                   [init_delta, init_log_sigma, init_u_pre_reparam, init_w, init_b],
                   [init_delta, init_log_sigma, init_mu_z, init_log_sigma_z]]

    HVAE_model_1 = HVAE(params,
        ['delta', 'log_sigma', 'logit_eps', 'log_T_0'],
        init_params[0], 
        'HVAE_1', d, params['HVAE_K_1'])
    HVAE_model_2 = HVAE(params,
        ['delta', 'log_sigma', 'logit_eps', 'log_T_0'],
        init_params[1], 
        'HVAE_2', d, params['HVAE_K_2'])
    
    
    NF_model_1 = NF(params,
        ['delta', 'log_sigma', 'u_pre_reparam', 'w', 'b'],
        init_params[2],
        'NF_1', d, params['NF_K_1'])
    
    NF_model_2 = NF(params,
        ['delta', 'log_sigma', 'u_pre_reparam', 'w', 'b'],
        init_params[3],
        'NF_2', d, params['NF_K_2'])
    
    VB_model = VB(params,
        ['delta', 'log_sigma', 'mu_z', 'log_sigma_z'], 
        init_params[4], 'VB', d)

    #model_list = [HVAE_model_1, HVAE_model_2, NF_model_1]
    #model_list = [HVAE_model_1, NF_model_1]
    model_list = [HVAE_model_1, HVAE_model_2, NF_model_1, NF_model_2, VB_model]

    print("Models were initialized!")
    
    ######### Generate Training Data & Save - One for each test

    train_data_list = []

    for i in range(params['n_tests']):
        z = MultivariateNormal(prior_mu_z, prior_sigma_z).sample()
        x = MultivariateNormal(z + t_delta, 
                               torch.diag(t_sigma**2)).sample([params['n_data']])
        x = x.to(device)
        train_data_list.append(x)

    # Folder should have already been created in the initializations
    data_path = os.path.join('save', str(d), 'train_data.p')
    pickle.dump(train_data_list, open(data_path, 'wb')) 

    # Store the final parameter values for all test runs in this dictionary
    final_params = {}

    for i in range(len(model_list)):
        final_values = []
        m = model_list[i]
        model_init_params = init_params[i]

        for j in range(params['n_tests']):
            print("------------------------------------------------------")
            print(f"Test = {j}")
            print("------------------------------------------------------")
            m._reinitialize(model_init_params)
            gaussian_data = gaussian_dataset(train_data_list[j])
            train_dataloader = DataLoader(gaussian_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
            (delta, sigma) = m.train(train_dataloader, j, 
                                     t_delta, t_sigma,
                                     clip_value)
            final_values.append((delta, sigma))

        final_params[m.model_name] = final_values.copy()

    ######### Test models using difference between parameters

    param_diffs = {}

    for m in model_list:
        
        diffs = []

        for i in range(params['n_tests']):
            delta = final_params[m.model_name][i][0]
            sigma = final_params[m.model_name][i][1]

            delta_diff = np.sum((delta - t_delta.numpy())**2)
            sigma_diff = np.sum((sigma - t_sigma.numpy())**2)
            theta_diff = delta_diff + sigma_diff
            print(f"Model {m.model_name}, test = {i}, result delta diff = {delta_diff}")
            print(f"Model {m.model_name}, test = {i}, result sigma diff = {sigma_diff}")
            print(f"Model {m.model_name}, test = {i}, result theta diff = {theta_diff}")

            diffs.append((delta_diff, sigma_diff, theta_diff))

        param_diffs[m.model_name] = diffs.copy()

    # Save parameter differences in a pickle file
    diff_path = os.path.join('save', str(d), 'all_diffs.p')
    pickle.dump(param_diffs, open(diff_path, 'wb'))

In [8]:
params = {

    # PROBLEM SPECIFICATION
    #'dims': [1, 2, 3, 5, 11, 25, 51, 101, 201, 301], # Dimensions to test
    'dims': [200],
    'n_data': 10000,                                 # Number of data points

    # TEST HYPERPARAMETERS
    'n_tests': 3,      # Number of experiments to run

    # GLOBAL OPTIMIZATION PARAMETERS
    'n_iter': 2000,    # Number of optimization iterations
    'n_batch': 10,      # Number of points for ELBO estimate
    'rms_eta': 0.001,   # Stepsize for RMSProp
    'save_every': 20,   # Save parameter information every so often
    'print_every': 20, # Print less often than save

    # HVAE HYPERPARAMETERS
    'HVAE_K_1': 1,    # Number of leapfrog/cooling steps for HVAE flow 1
    'HVAE_K_2': 10,   # Number of leapfrog/cooling steps for HVAE flow 2
    'max_eps': 0.5,   # Maximum leapfrog step size per dimension 

    # NF HYPERPARAMETERS
    'NF_K_1': 1,    # Number of flow steps for NF flow 1
    'NF_K_2': 30,   # Number of flow steps for NF flow 2
    
}

In [9]:
device = torch.device("cpu")
dtype = torch.float32
clip_value = 1e-4
batch_size = 256

In [10]:
dims = params['dims']
seed = 12345
np.random.seed(seed)
torch.manual_seed(seed)
for d in dims:
    print('**** Running test for d={0:d} ****'.format(d))
    run_test(d, params, batch_size, clip_value, dtype, device)

Выходные данные были обрезаны до нескольких последних строк (5000).
HVAE_2, d: 200, Iter: 3-420, s/iter: 2.636e-01, ELBO: -1.087e+06
Cur diff delta = 368.30657958984375
Cur diff sigma = 0.7167985439300537
Cur diff theta = 369.0233781337738
---------------------------------------
HVAE_2, d: 200, Iter: 3-440, s/iter: 2.505e-01, ELBO: -1.077e+06
Cur diff delta = 283.0844421386719
Cur diff sigma = 0.4775009751319885
Cur diff theta = 283.56194311380386
---------------------------------------
HVAE_2, d: 200, Iter: 3-460, s/iter: 2.545e-01, ELBO: -1.067e+06
Cur diff delta = 232.56497192382812
Cur diff sigma = 0.3695141673088074
Cur diff theta = 232.93448609113693
---------------------------------------
HVAE_2, d: 200, Iter: 3-480, s/iter: 2.537e-01, ELBO: -1.062e+06
Cur diff delta = 207.742431640625
Cur diff sigma = 0.31886768341064453
Cur diff theta = 208.06129932403564
---------------------------------------
HVAE_2, d: 200, Iter: 3-500, s/iter: 2.807e-01, ELBO: -1.059e+06
Cur diff delta = 1

In [11]:
model_names = ['HVAE_1', 'HVAE_2', 'NF_1', 'NF_2', 'VB']
diff_delta_results = {}
diff_sigma_results = {}
diff_results = {}
for name in model_names:
    diff_delta_tests = np.zeros(params['n_tests'])
    diff_sigma_tests = np.zeros(params['n_tests'])
    diff_tests = np.zeros(params['n_tests'])
    for i in range(params['n_tests']):
        file_name = name + '_train_' + str(i) + '.p'
        diff_path = os.path.join('save', str(params['dims'][0]), file_name)
        with open(diff_path, 'rb') as f:
            data = pickle.load(f)
            elbo = data['elbo']
            diff_delta = data['diff_delta']
            diff_sigma = data['diff_sigma']
            diff = diff_delta + diff_sigma
            diff_delta_is_not_nan = diff_delta[~np.isnan(diff_delta)]
            diff_sigma_is_not_nan = diff_sigma[~np.isnan(diff_sigma)]
            diff_is_not_nan = diff[~np.isnan(diff)]
            diff_delta_tests[i] = diff_delta_is_not_nan[-1]
            diff_sigma_tests[i] = diff_sigma_is_not_nan[-1]
            diff_tests[i] = diff_is_not_nan[-1]
    diff_delta_results[name] = diff_delta_tests
    diff_sigma_results[name] = diff_sigma_tests
    diff_results[name] = diff_tests
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff delta = {diff_delta_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff sigma = {diff_sigma_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff = {diff_tests}")

Model = HVAE_1, dimensionality = 200, diff delta = [183.45848083 205.34617615 200.65716553]
Model = HVAE_1, dimensionality = 200, diff sigma = [0.33295846 0.33400047 0.33786768]
Model = HVAE_1, dimensionality = 200, diff = [183.79144287 205.68017578 200.99502563]
Model = HVAE_2, dimensionality = 200, diff delta = [183.79724121 205.11697388 200.20713806]
Model = HVAE_2, dimensionality = 200, diff sigma = [0.30696803 0.30571064 0.31064045]
Model = HVAE_2, dimensionality = 200, diff = [184.10420227 205.42268372 200.51777649]
Model = NF_1, dimensionality = 200, diff delta = [183.93060303 205.45214844 200.53007507]
Model = NF_1, dimensionality = 200, diff sigma = [125.79404449 125.87238312 125.91531372]
Model = NF_1, dimensionality = 200, diff = [309.72463989 331.32452393 326.44537354]
Model = NF_2, dimensionality = 200, diff delta = [184.4203186  205.47422791 200.65820312]
Model = NF_2, dimensionality = 200, diff sigma = [125.53786469 125.62741852 125.10678101]
Model = NF_2, dimensionality

In [13]:
for name in model_names:
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff delta = {diff_delta_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff sigma = {diff_sigma_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff = {diff_results[name].mean()}")

Model = HVAE_1, dimensionality = 200, mean diff delta = 196.48727416992188
Model = HVAE_1, dimensionality = 200, mean diff sigma = 0.3349422017733256
Model = HVAE_1, dimensionality = 200, mean diff = 196.82221476236978
Model = HVAE_2, dimensionality = 200, mean diff delta = 196.37378438313803
Model = HVAE_2, dimensionality = 200, mean diff sigma = 0.30777304371198017
Model = HVAE_2, dimensionality = 200, mean diff = 196.68155415852866
Model = NF_1, dimensionality = 200, mean diff delta = 196.63760884602866
Model = NF_1, dimensionality = 200, mean diff sigma = 125.86058044433594
Model = NF_1, dimensionality = 200, mean diff = 322.49817911783856
Model = NF_2, dimensionality = 200, mean diff delta = 196.85091654459634
Model = NF_2, dimensionality = 200, mean diff sigma = 125.4240214029948
Model = NF_2, dimensionality = 200, mean diff = 322.2749430338542
Model = VB, dimensionality = 200, mean diff delta = 635.1636759440104
Model = VB, dimensionality = 200, mean diff sigma = 0.0034049580184

In [24]:
params = {

    # PROBLEM SPECIFICATION
    #'dims': [1, 2, 3, 5, 11, 25, 51, 101, 201, 301], # Dimensions to test
    'dims': [200],
    'n_data': 10000,                                 # Number of data points

    # TEST HYPERPARAMETERS
    'n_tests': 3,      # Number of experiments to run

    # GLOBAL OPTIMIZATION PARAMETERS
    'n_iter': 4000,    # Number of optimization iterations
    'n_batch': 10,      # Number of points for ELBO estimate
    'rms_eta': 0.001,   # Stepsize for RMSProp
    'save_every': 20,   # Save parameter information every so often
    'print_every': 20, # Print less often than save

    # HVAE HYPERPARAMETERS
    'HVAE_K_1': 1,    # Number of leapfrog/cooling steps for HVAE flow 1
    'HVAE_K_2': 10,   # Number of leapfrog/cooling steps for HVAE flow 2
    'max_eps': 0.5,   # Maximum leapfrog step size per dimension 

    # NF HYPERPARAMETERS
    'NF_K_1': 1,    # Number of flow steps for NF flow 1
    'NF_K_2': 30,   # Number of flow steps for NF flow 2
    
}

In [25]:
def run_test(d, params, 
             batch_size=256,
             clip_value=None,
             dtype=torch.float32, 
             device=torch.device("cpu")):
    """ 
    Run the gaussian test with dimension d 
    """

    ######### Problem Specification

    # Data generation parameters
    prior_mu_z = torch.zeros(d, dtype=dtype)    # Prior mean
    prior_sigma_z = torch.eye(d, dtype=dtype)   # Prior covariance matrix

    # True model parameters
    num_range = torch.arange(-(d-1)/2, (d+1)/2, dtype=dtype)

    t_delta =  num_range / 5 

    print(f"true delta = {t_delta.numpy()}")

    if d == 1:
        t_sigma = torch.ones(1, dtype=dtype)
    else: 
        # Allow sigma to range from 0.1 to 1
        t_sigma = 36/(10*(d-1)**2) * num_range**2 + 0.1 
    print(f"true sigma = {t_sigma.numpy()}")

    ######### Variable Initialization

    # Initial model parameters - same across all methods
    init_delta = prior_mu_z.clone().to(device)
    init_log_sigma = 3 * torch.ones(d).to(device)

    # Initial HVAE variational parameters
    init_T = torch.tensor(5., dtype=dtype, device=device)
    init_eps = 0.005 * torch.ones(d, dtype=dtype)
    max_eps = params['max_eps'] * torch.ones(d, dtype=dtype)
    init_logit_eps = torch.log(init_eps/(max_eps - init_eps)).to(device)
    init_log_T_0 = torch.log(init_T - 1)

    # Initial NF variational parameters
    init_u_pre_reparam = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, 
                                                                scale=0.1, 
                                                                size=d),
                                      dtype=dtype, device=device)
    init_w = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, scale=0.1, size=d),
                          dtype=dtype, device=device)
    init_b = torch.tensor(0.1, dtype=dtype, device=device)

    # Initial VAE parameters
    init_mu_z = prior_mu_z.clone()
    init_log_sigma_z = torch.ones(d, dtype=dtype)

    ######### Set up models
    init_params = [[init_delta, init_log_sigma, init_logit_eps, init_log_T_0],
                   [init_delta, init_log_sigma, init_logit_eps, init_log_T_0],
                   [init_delta, init_log_sigma, init_u_pre_reparam, init_w, init_b],
                   [init_delta, init_log_sigma, init_u_pre_reparam, init_w, init_b],
                   [init_delta, init_log_sigma, init_mu_z, init_log_sigma_z]]
    
    VB_model = VB(params,
        ['delta', 'log_sigma', 'mu_z', 'log_sigma_z'], 
        init_params[4], 'VB', d)

    #model_list = [HVAE_model_1, HVAE_model_2, NF_model_1]
    #model_list = [HVAE_model_1, NF_model_1]
    model_list = [VB_model]

    print("Models were initialized!")
    
    ######### Generate Training Data & Save - One for each test

    train_data_list = []

    for i in range(params['n_tests']):
        z = MultivariateNormal(prior_mu_z, prior_sigma_z).sample()
        x = MultivariateNormal(z + t_delta, 
                               torch.diag(t_sigma**2)).sample([params['n_data']])
        x = x.to(device)
        train_data_list.append(x)

    # Folder should have already been created in the initializations
    data_path = os.path.join('save', str(d), 'train_data.p')
    pickle.dump(train_data_list, open(data_path, 'wb')) 

    # Store the final parameter values for all test runs in this dictionary
    final_params = {}

    for i in range(len(model_list)):
        final_values = []
        m = model_list[i]
        model_init_params = init_params[i]

        for j in range(params['n_tests']):
            print("------------------------------------------------------")
            print(f"Test = {j}")
            print("------------------------------------------------------")
            m._reinitialize(model_init_params)
            gaussian_data = gaussian_dataset(train_data_list[j])
            train_dataloader = DataLoader(gaussian_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
            (delta, sigma) = m.train(train_dataloader, j, 
                                     t_delta, t_sigma,
                                     clip_value)
            final_values.append((delta, sigma))

        final_params[m.model_name] = final_values.copy()

    ######### Test models using difference between parameters

    param_diffs = {}

    for m in model_list:
        
        diffs = []

        for i in range(params['n_tests']):
            delta = final_params[m.model_name][i][0]
            sigma = final_params[m.model_name][i][1]

            delta_diff = np.sum((delta - t_delta.numpy())**2)
            sigma_diff = np.sum((sigma - t_sigma.numpy())**2)
            theta_diff = delta_diff + sigma_diff
            print(f"Model {m.model_name}, test = {i}, result delta diff = {delta_diff}")
            print(f"Model {m.model_name}, test = {i}, result sigma diff = {sigma_diff}")
            print(f"Model {m.model_name}, test = {i}, result theta diff = {theta_diff}")

            diffs.append((delta_diff, sigma_diff, theta_diff))

        param_diffs[m.model_name] = diffs.copy()

    # Save parameter differences in a pickle file
    diff_path = os.path.join('save', str(d), 'all_diffs.p')
    pickle.dump(param_diffs, open(diff_path, 'wb'))

In [27]:
device = torch.device("cuda")
dtype = torch.float32
clip_value = None
batch_size = 256

In [28]:
dims = params['dims']
seed = 12345
np.random.seed(seed)
torch.manual_seed(seed)
for d in dims:
    print('**** Running test for d={0:d} ****'.format(d))
    run_test(d, params, batch_size, clip_value, dtype, device)

**** Running test for d=200 ****
true delta = [-19.9 -19.7 -19.5 -19.3 -19.1 -18.9 -18.7 -18.5 -18.3 -18.1 -17.9 -17.7
 -17.5 -17.3 -17.1 -16.9 -16.7 -16.5 -16.3 -16.1 -15.9 -15.7 -15.5 -15.3
 -15.1 -14.9 -14.7 -14.5 -14.3 -14.1 -13.9 -13.7 -13.5 -13.3 -13.1 -12.9
 -12.7 -12.5 -12.3 -12.1 -11.9 -11.7 -11.5 -11.3 -11.1 -10.9 -10.7 -10.5
 -10.3 -10.1  -9.9  -9.7  -9.5  -9.3  -9.1  -8.9  -8.7  -8.5  -8.3  -8.1
  -7.9  -7.7  -7.5  -7.3  -7.1  -6.9  -6.7  -6.5  -6.3  -6.1  -5.9  -5.7
  -5.5  -5.3  -5.1  -4.9  -4.7  -4.5  -4.3  -4.1  -3.9  -3.7  -3.5  -3.3
  -3.1  -2.9  -2.7  -2.5  -2.3  -2.1  -1.9  -1.7  -1.5  -1.3  -1.1  -0.9
  -0.7  -0.5  -0.3  -0.1   0.1   0.3   0.5   0.7   0.9   1.1   1.3   1.5
   1.7   1.9   2.1   2.3   2.5   2.7   2.9   3.1   3.3   3.5   3.7   3.9
   4.1   4.3   4.5   4.7   4.9   5.1   5.3   5.5   5.7   5.9   6.1   6.3
   6.5   6.7   6.9   7.1   7.3   7.5   7.7   7.9   8.1   8.3   8.5   8.7
   8.9   9.1   9.3   9.5   9.7   9.9  10.1  10.3  10.5  10.7  10.9  11.1
  11.

In [30]:
model_names = ['HVAE_1', 'HVAE_2', 'NF_1', 'NF_2', 'VB']
diff_delta_results = {}
diff_sigma_results = {}
diff_results = {}
for name in model_names:
    diff_delta_tests = np.zeros(params['n_tests'])
    diff_sigma_tests = np.zeros(params['n_tests'])
    diff_tests = np.zeros(params['n_tests'])
    for i in range(params['n_tests']):
        file_name = name + '_train_' + str(i) + '.p'
        diff_path = os.path.join('save', str(params['dims'][0]), file_name)
        with open(diff_path, 'rb') as f:
            data = pickle.load(f)
            elbo = data['elbo']
            diff_delta = data['diff_delta']
            diff_sigma = data['diff_sigma']
            diff = diff_delta + diff_sigma
            diff_delta_is_not_nan = diff_delta[~np.isnan(diff_delta)]
            diff_sigma_is_not_nan = diff_sigma[~np.isnan(diff_sigma)]
            diff_is_not_nan = diff[~np.isnan(diff)]
            diff_delta_tests[i] = diff_delta_is_not_nan[-1]
            diff_sigma_tests[i] = diff_sigma_is_not_nan[-1]
            diff_tests[i] = diff_is_not_nan[-1]
    diff_delta_results[name] = diff_delta_tests
    diff_sigma_results[name] = diff_sigma_tests
    diff_results[name] = diff_tests
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff delta = {diff_delta_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff sigma = {diff_sigma_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff = {diff_tests}")

Model = HVAE_1, dimensionality = 200, diff delta = [183.45848083 205.34617615 200.65716553]
Model = HVAE_1, dimensionality = 200, diff sigma = [0.33295846 0.33400047 0.33786768]
Model = HVAE_1, dimensionality = 200, diff = [183.79144287 205.68017578 200.99502563]
Model = HVAE_2, dimensionality = 200, diff delta = [183.79724121 205.11697388 200.20713806]
Model = HVAE_2, dimensionality = 200, diff sigma = [0.30696803 0.30571064 0.31064045]
Model = HVAE_2, dimensionality = 200, diff = [184.10420227 205.42268372 200.51777649]
Model = NF_1, dimensionality = 200, diff delta = [183.93060303 205.45214844 200.53007507]
Model = NF_1, dimensionality = 200, diff sigma = [125.79404449 125.87238312 125.91531372]
Model = NF_1, dimensionality = 200, diff = [309.72463989 331.32452393 326.44537354]
Model = NF_2, dimensionality = 200, diff delta = [184.4203186  205.47422791 200.65820312]
Model = NF_2, dimensionality = 200, diff sigma = [125.53786469 125.62741852 125.10678101]
Model = NF_2, dimensionality

In [31]:
for name in model_names:
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff delta = {diff_delta_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff sigma = {diff_sigma_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff = {diff_results[name].mean()}")

Model = HVAE_1, dimensionality = 200, mean diff delta = 196.48727416992188
Model = HVAE_1, dimensionality = 200, mean diff sigma = 0.3349422017733256
Model = HVAE_1, dimensionality = 200, mean diff = 196.82221476236978
Model = HVAE_2, dimensionality = 200, mean diff delta = 196.37378438313803
Model = HVAE_2, dimensionality = 200, mean diff sigma = 0.30777304371198017
Model = HVAE_2, dimensionality = 200, mean diff = 196.68155415852866
Model = NF_1, dimensionality = 200, mean diff delta = 196.63760884602866
Model = NF_1, dimensionality = 200, mean diff sigma = 125.86058044433594
Model = NF_1, dimensionality = 200, mean diff = 322.49817911783856
Model = NF_2, dimensionality = 200, mean diff delta = 196.85091654459634
Model = NF_2, dimensionality = 200, mean diff sigma = 125.4240214029948
Model = NF_2, dimensionality = 200, mean diff = 322.2749430338542
Model = VB, dimensionality = 200, mean diff delta = 270.1203308105469
Model = VB, dimensionality = 200, mean diff sigma = 0.0029109336125

In [32]:
def run_test(d, params, 
             batch_size=256,
             clip_value=None,
             dtype=torch.float32, 
             device=torch.device("cpu")):
    """ 
    Run the gaussian test with dimension d 
    """

    ######### Problem Specification

    # Data generation parameters
    prior_mu_z = torch.zeros(d, dtype=dtype)    # Prior mean
    prior_sigma_z = torch.eye(d, dtype=dtype)   # Prior covariance matrix

    # True model parameters
    num_range = torch.arange(-(d-1)/2, (d+1)/2, dtype=dtype)

    t_delta =  num_range / 5 

    print(f"true delta = {t_delta.numpy()}")

    if d == 1:
        t_sigma = torch.ones(1, dtype=dtype)
    else: 
        # Allow sigma to range from 0.1 to 1
        t_sigma = 36/(10*(d-1)**2) * num_range**2 + 0.1 
    print(f"true sigma = {t_sigma.numpy()}")

    ######### Variable Initialization

    # Initial model parameters - same across all methods
    init_delta = prior_mu_z.clone().to(device)
    init_log_sigma = 3 * torch.ones(d).to(device)

    # Initial HVAE variational parameters
    init_T = torch.tensor(5., dtype=dtype, device=device)
    init_eps = 0.005 * torch.ones(d, dtype=dtype)
    max_eps = params['max_eps'] * torch.ones(d, dtype=dtype)
    init_logit_eps = torch.log(init_eps/(max_eps - init_eps)).to(device)
    init_log_T_0 = torch.log(init_T - 1)

    # Initial NF variational parameters
    init_u_pre_reparam = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, 
                                                                scale=0.1, 
                                                                size=d),
                                      dtype=dtype, device=device)
    init_w = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, scale=0.1, size=d),
                          dtype=dtype, device=device)
    init_b = torch.tensor(0.1, dtype=dtype, device=device)

    # Initial VAE parameters
    init_mu_z = prior_mu_z.clone()
    init_log_sigma_z = torch.ones(d, dtype=dtype)

    ######### Set up models
    init_params = [[init_delta, init_log_sigma, init_logit_eps],
                   [init_delta, init_log_sigma, init_logit_eps]]
    
    HVAE_model_notemp_1 = HVAE(params,
        ['delta', 'log_sigma', 'logit_eps'],
        init_params[0], 
        'HVAE_notemp_1', d, params['HVAE_K_1'])
    HVAE_model_notemp_2 = HVAE(params,
        ['delta', 'log_sigma', 'logit_eps'], 
        init_params[1],
        'HVAE_notemp_2', d, params['HVAE_K_2'])

    #model_list = [HVAE_model_1, HVAE_model_2, NF_model_1]
    #model_list = [HVAE_model_1, NF_model_1]
    model_list = [HVAE_model_notemp_1, HVAE_model_notemp_2]

    print("Models were initialized!")
    
    ######### Generate Training Data & Save - One for each test

    train_data_list = []

    for i in range(params['n_tests']):
        z = MultivariateNormal(prior_mu_z, prior_sigma_z).sample()
        x = MultivariateNormal(z + t_delta, 
                               torch.diag(t_sigma**2)).sample([params['n_data']])
        x = x.to(device)
        train_data_list.append(x)

    # Folder should have already been created in the initializations
    data_path = os.path.join('save', str(d), 'train_data.p')
    pickle.dump(train_data_list, open(data_path, 'wb')) 

    # Store the final parameter values for all test runs in this dictionary
    final_params = {}

    for i in range(len(model_list)):
        final_values = []
        m = model_list[i]
        model_init_params = init_params[i]

        for j in range(params['n_tests']):
            print("------------------------------------------------------")
            print(f"Test = {j}")
            print("------------------------------------------------------")
            m._reinitialize(model_init_params)
            gaussian_data = gaussian_dataset(train_data_list[j])
            train_dataloader = DataLoader(gaussian_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
            (delta, sigma) = m.train(train_dataloader, j, 
                                     t_delta, t_sigma,
                                     clip_value)
            final_values.append((delta, sigma))

        final_params[m.model_name] = final_values.copy()

    ######### Test models using difference between parameters

    param_diffs = {}

    for m in model_list:
        
        diffs = []

        for i in range(params['n_tests']):
            delta = final_params[m.model_name][i][0]
            sigma = final_params[m.model_name][i][1]

            delta_diff = np.sum((delta - t_delta.numpy())**2)
            sigma_diff = np.sum((sigma - t_sigma.numpy())**2)
            theta_diff = delta_diff + sigma_diff
            print(f"Model {m.model_name}, test = {i}, result delta diff = {delta_diff}")
            print(f"Model {m.model_name}, test = {i}, result sigma diff = {sigma_diff}")
            print(f"Model {m.model_name}, test = {i}, result theta diff = {theta_diff}")

            diffs.append((delta_diff, sigma_diff, theta_diff))

        param_diffs[m.model_name] = diffs.copy()

    # Save parameter differences in a pickle file
    diff_path = os.path.join('save', str(d), 'all_diffs.p')
    pickle.dump(param_diffs, open(diff_path, 'wb'))

In [33]:
device = torch.device("cuda")
dtype = torch.float32
clip_value = None
batch_size = 256

In [34]:
params = {

    # PROBLEM SPECIFICATION
    #'dims': [1, 2, 3, 5, 11, 25, 51, 101, 201, 301], # Dimensions to test
    'dims': [200],
    'n_data': 10000,                                 # Number of data points

    # TEST HYPERPARAMETERS
    'n_tests': 3,      # Number of experiments to run

    # GLOBAL OPTIMIZATION PARAMETERS
    'n_iter': 2000,    # Number of optimization iterations
    'n_batch': 10,      # Number of points for ELBO estimate
    'rms_eta': 0.001,   # Stepsize for RMSProp
    'save_every': 20,   # Save parameter information every so often
    'print_every': 20, # Print less often than save

    # HVAE HYPERPARAMETERS
    'HVAE_K_1': 1,    # Number of leapfrog/cooling steps for HVAE flow 1
    'HVAE_K_2': 10,   # Number of leapfrog/cooling steps for HVAE flow 2
    'max_eps': 0.5,   # Maximum leapfrog step size per dimension 

    # NF HYPERPARAMETERS
    'NF_K_1': 1,    # Number of flow steps for NF flow 1
    'NF_K_2': 30,   # Number of flow steps for NF flow 2
    
}

In [35]:
dims = params['dims']
seed = 12345
np.random.seed(seed)
torch.manual_seed(seed)
for d in dims:
    print('**** Running test for d={0:d} ****'.format(d))
    run_test(d, params, batch_size, clip_value, dtype, device)

**** Running test for d=200 ****
true delta = [-19.9 -19.7 -19.5 -19.3 -19.1 -18.9 -18.7 -18.5 -18.3 -18.1 -17.9 -17.7
 -17.5 -17.3 -17.1 -16.9 -16.7 -16.5 -16.3 -16.1 -15.9 -15.7 -15.5 -15.3
 -15.1 -14.9 -14.7 -14.5 -14.3 -14.1 -13.9 -13.7 -13.5 -13.3 -13.1 -12.9
 -12.7 -12.5 -12.3 -12.1 -11.9 -11.7 -11.5 -11.3 -11.1 -10.9 -10.7 -10.5
 -10.3 -10.1  -9.9  -9.7  -9.5  -9.3  -9.1  -8.9  -8.7  -8.5  -8.3  -8.1
  -7.9  -7.7  -7.5  -7.3  -7.1  -6.9  -6.7  -6.5  -6.3  -6.1  -5.9  -5.7
  -5.5  -5.3  -5.1  -4.9  -4.7  -4.5  -4.3  -4.1  -3.9  -3.7  -3.5  -3.3
  -3.1  -2.9  -2.7  -2.5  -2.3  -2.1  -1.9  -1.7  -1.5  -1.3  -1.1  -0.9
  -0.7  -0.5  -0.3  -0.1   0.1   0.3   0.5   0.7   0.9   1.1   1.3   1.5
   1.7   1.9   2.1   2.3   2.5   2.7   2.9   3.1   3.3   3.5   3.7   3.9
   4.1   4.3   4.5   4.7   4.9   5.1   5.3   5.5   5.7   5.9   6.1   6.3
   6.5   6.7   6.9   7.1   7.3   7.5   7.7   7.9   8.1   8.3   8.5   8.7
   8.9   9.1   9.3   9.5   9.7   9.9  10.1  10.3  10.5  10.7  10.9  11.1
  11.

In [36]:
model_names = ['HVAE_notemp_1', 'HVAE_notemp_2']
diff_delta_results = {}
diff_sigma_results = {}
diff_results = {}
for name in model_names:
    diff_delta_tests = np.zeros(params['n_tests'])
    diff_sigma_tests = np.zeros(params['n_tests'])
    diff_tests = np.zeros(params['n_tests'])
    for i in range(params['n_tests']):
        file_name = name + '_train_' + str(i) + '.p'
        diff_path = os.path.join('save', str(params['dims'][0]), file_name)
        with open(diff_path, 'rb') as f:
            data = pickle.load(f)
            elbo = data['elbo']
            diff_delta = data['diff_delta']
            diff_sigma = data['diff_sigma']
            diff = diff_delta + diff_sigma
            diff_delta_is_not_nan = diff_delta[~np.isnan(diff_delta)]
            diff_sigma_is_not_nan = diff_sigma[~np.isnan(diff_sigma)]
            diff_is_not_nan = diff[~np.isnan(diff)]
            diff_delta_tests[i] = diff_delta_is_not_nan[-1]
            diff_sigma_tests[i] = diff_sigma_is_not_nan[-1]
            diff_tests[i] = diff_is_not_nan[-1]
    diff_delta_results[name] = diff_delta_tests
    diff_sigma_results[name] = diff_sigma_tests
    diff_results[name] = diff_tests
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff delta = {diff_delta_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff sigma = {diff_sigma_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff = {diff_tests}")

Model = HVAE_notemp_1, dimensionality = 200, diff delta = [183.73350525 205.05963135 200.19509888]
Model = HVAE_notemp_1, dimensionality = 200, diff sigma = [32.31481934 32.25682831 32.26351929]
Model = HVAE_notemp_1, dimensionality = 200, diff = [216.04832458 237.31646729 232.45861816]
Model = HVAE_notemp_2, dimensionality = 200, diff delta = [183.50814819 204.95404053 200.06770325]
Model = HVAE_notemp_2, dimensionality = 200, diff sigma = [102.8013382  102.28044128 102.77435303]
Model = HVAE_notemp_2, dimensionality = 200, diff = [286.30947876 307.23449707 302.84204102]


In [37]:
for name in model_names:
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff delta = {diff_delta_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff sigma = {diff_sigma_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff = {diff_results[name].mean()}")

Model = HVAE_notemp_1, dimensionality = 200, mean diff delta = 196.32941182454428
Model = HVAE_notemp_1, dimensionality = 200, mean diff sigma = 32.27838897705078
Model = HVAE_notemp_1, dimensionality = 200, mean diff = 228.60780334472656
Model = HVAE_notemp_2, dimensionality = 200, mean diff delta = 196.17663065592447
Model = HVAE_notemp_2, dimensionality = 200, mean diff sigma = 102.61871083577473
Model = HVAE_notemp_2, dimensionality = 200, mean diff = 298.7953389485677


In [38]:
def run_test(d, params, 
             batch_size=256,
             clip_value=None,
             dtype=torch.float32, 
             device=torch.device("cpu")):
    """ 
    Run the gaussian test with dimension d 
    """

    ######### Problem Specification

    # Data generation parameters
    prior_mu_z = torch.zeros(d, dtype=dtype)    # Prior mean
    prior_sigma_z = torch.eye(d, dtype=dtype)   # Prior covariance matrix

    # True model parameters
    num_range = torch.arange(-(d-1)/2, (d+1)/2, dtype=dtype)

    t_delta =  num_range / 5 

    print(f"true delta = {t_delta.numpy()}")

    if d == 1:
        t_sigma = torch.ones(1, dtype=dtype)
    else: 
        # Allow sigma to range from 0.1 to 1
        t_sigma = 36/(10*(d-1)**2) * num_range**2 + 0.1 
    print(f"true sigma = {t_sigma.numpy()}")

    ######### Variable Initialization

    # Initial model parameters - same across all methods
    init_delta = prior_mu_z.clone().to(device)
    init_log_sigma = 3 * torch.ones(d).to(device)

    # Initial HVAE variational parameters
    init_T = torch.tensor(5., dtype=dtype, device=device)
    init_eps = 0.005 * torch.ones(d, dtype=dtype)
    max_eps = params['max_eps'] * torch.ones(d, dtype=dtype)
    init_logit_eps = torch.log(init_eps/(max_eps - init_eps)).to(device)
    init_log_T_0 = torch.log(init_T - 1)

    # Initial NF variational parameters
    init_u_pre_reparam = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, 
                                                                scale=0.1, 
                                                                size=d),
                                      dtype=dtype, device=device)
    init_w = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, scale=0.1, size=d),
                          dtype=dtype, device=device)
    init_b = torch.tensor(0.1, dtype=dtype, device=device)

    # Initial VAE parameters
    init_mu_z = prior_mu_z.clone()
    init_log_sigma_z = torch.ones(d, dtype=dtype)

    ######### Set up models
    init_params = [[init_delta, init_log_sigma, init_logit_eps, init_log_T_0],
                   [init_delta, init_log_sigma, init_logit_eps, init_log_T_0],
                   [init_delta, init_log_sigma, init_u_pre_reparam, init_w, init_b],
                   [init_delta, init_log_sigma, init_u_pre_reparam, init_w, init_b],
                   [init_delta, init_log_sigma, init_mu_z, init_log_sigma_z]]

    VB_model = VB(params,
        ['delta', 'log_sigma', 'mu_z', 'log_sigma_z'], 
        init_params[4], 'VB', d)

    #model_list = [HVAE_model_1, HVAE_model_2, NF_model_1]
    #model_list = [HVAE_model_1, NF_model_1]
    model_list = [VB_model]

    print("Models were initialized!")
    
    ######### Generate Training Data & Save - One for each test

    train_data_list = []

    for i in range(params['n_tests']):
        z = MultivariateNormal(prior_mu_z, prior_sigma_z).sample()
        x = MultivariateNormal(z + t_delta, 
                               torch.diag(t_sigma**2)).sample([params['n_data']])
        x = x.to(device)
        train_data_list.append(x)

    # Folder should have already been created in the initializations
    data_path = os.path.join('save', str(d), 'train_data.p')
    pickle.dump(train_data_list, open(data_path, 'wb')) 

    # Store the final parameter values for all test runs in this dictionary
    final_params = {}

    for i in range(len(model_list)):
        final_values = []
        m = model_list[i]
        model_init_params = init_params[i]

        for j in range(params['n_tests']):
            print("------------------------------------------------------")
            print(f"Test = {j}")
            print("------------------------------------------------------")
            m._reinitialize(model_init_params)
            gaussian_data = gaussian_dataset(train_data_list[j])
            train_dataloader = DataLoader(gaussian_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
            (delta, sigma) = m.train(train_dataloader, j, 
                                     t_delta, t_sigma,
                                     clip_value)
            final_values.append((delta, sigma))

        final_params[m.model_name] = final_values.copy()

    ######### Test models using difference between parameters

    param_diffs = {}

    for m in model_list:
        
        diffs = []

        for i in range(params['n_tests']):
            delta = final_params[m.model_name][i][0]
            sigma = final_params[m.model_name][i][1]

            delta_diff = np.sum((delta - t_delta.numpy())**2)
            sigma_diff = np.sum((sigma - t_sigma.numpy())**2)
            theta_diff = delta_diff + sigma_diff
            print(f"Model {m.model_name}, test = {i}, result delta diff = {delta_diff}")
            print(f"Model {m.model_name}, test = {i}, result sigma diff = {sigma_diff}")
            print(f"Model {m.model_name}, test = {i}, result theta diff = {theta_diff}")

            diffs.append((delta_diff, sigma_diff, theta_diff))

        param_diffs[m.model_name] = diffs.copy()

    # Save parameter differences in a pickle file
    diff_path = os.path.join('save', str(d), 'all_diffs.p')
    pickle.dump(param_diffs, open(diff_path, 'wb'))

In [39]:
device = torch.device("cuda")
dtype = torch.float32
clip_value = None
batch_size = 256

In [40]:
params = {

    # PROBLEM SPECIFICATION
    #'dims': [1, 2, 3, 5, 11, 25, 51, 101, 201, 301], # Dimensions to test
    'dims': [400],
    'n_data': 10000,                                 # Number of data points

    # TEST HYPERPARAMETERS
    'n_tests': 3,      # Number of experiments to run

    # GLOBAL OPTIMIZATION PARAMETERS
    'n_iter': 5000,    # Number of optimization iterations
    'n_batch': 10,      # Number of points for ELBO estimate
    'rms_eta': 0.001,   # Stepsize for RMSProp
    'save_every': 20,   # Save parameter information every so often
    'print_every': 20, # Print less often than save

    # HVAE HYPERPARAMETERS
    'HVAE_K_1': 1,    # Number of leapfrog/cooling steps for HVAE flow 1
    'HVAE_K_2': 10,   # Number of leapfrog/cooling steps for HVAE flow 2
    'max_eps': 0.5,   # Maximum leapfrog step size per dimension 

    # NF HYPERPARAMETERS
    'NF_K_1': 1,    # Number of flow steps for NF flow 1
    'NF_K_2': 30,   # Number of flow steps for NF flow 2
    
}

In [41]:
dims = params['dims']
seed = 12345
np.random.seed(seed)
torch.manual_seed(seed)
for d in dims:
    print('**** Running test for d={0:d} ****'.format(d))
    run_test(d, params, batch_size, clip_value, dtype, device)

**** Running test for d=400 ****
true delta = [-39.9 -39.7 -39.5 -39.3 -39.1 -38.9 -38.7 -38.5 -38.3 -38.1 -37.9 -37.7
 -37.5 -37.3 -37.1 -36.9 -36.7 -36.5 -36.3 -36.1 -35.9 -35.7 -35.5 -35.3
 -35.1 -34.9 -34.7 -34.5 -34.3 -34.1 -33.9 -33.7 -33.5 -33.3 -33.1 -32.9
 -32.7 -32.5 -32.3 -32.1 -31.9 -31.7 -31.5 -31.3 -31.1 -30.9 -30.7 -30.5
 -30.3 -30.1 -29.9 -29.7 -29.5 -29.3 -29.1 -28.9 -28.7 -28.5 -28.3 -28.1
 -27.9 -27.7 -27.5 -27.3 -27.1 -26.9 -26.7 -26.5 -26.3 -26.1 -25.9 -25.7
 -25.5 -25.3 -25.1 -24.9 -24.7 -24.5 -24.3 -24.1 -23.9 -23.7 -23.5 -23.3
 -23.1 -22.9 -22.7 -22.5 -22.3 -22.1 -21.9 -21.7 -21.5 -21.3 -21.1 -20.9
 -20.7 -20.5 -20.3 -20.1 -19.9 -19.7 -19.5 -19.3 -19.1 -18.9 -18.7 -18.5
 -18.3 -18.1 -17.9 -17.7 -17.5 -17.3 -17.1 -16.9 -16.7 -16.5 -16.3 -16.1
 -15.9 -15.7 -15.5 -15.3 -15.1 -14.9 -14.7 -14.5 -14.3 -14.1 -13.9 -13.7
 -13.5 -13.3 -13.1 -12.9 -12.7 -12.5 -12.3 -12.1 -11.9 -11.7 -11.5 -11.3
 -11.1 -10.9 -10.7 -10.5 -10.3 -10.1  -9.9  -9.7  -9.5  -9.3  -9.1  -8.9
  -8.

In [44]:
model_names = ['VB']
diff_delta_results = {}
diff_sigma_results = {}
diff_results = {}
for name in model_names:
    diff_delta_tests = np.zeros(params['n_tests'])
    diff_sigma_tests = np.zeros(params['n_tests'])
    diff_tests = np.zeros(params['n_tests'])
    for i in range(params['n_tests']):
        file_name = name + '_train_' + str(i) + '.p'
        diff_path = os.path.join('save', str(params['dims'][0]), file_name)
        with open(diff_path, 'rb') as f:
            data = pickle.load(f)
            elbo = data['elbo']
            diff_delta = data['diff_delta']
            diff_sigma = data['diff_sigma']
            diff = diff_delta + diff_sigma
            diff_delta_is_not_nan = diff_delta[~np.isnan(diff_delta)]
            diff_sigma_is_not_nan = diff_sigma[~np.isnan(diff_sigma)]
            diff_is_not_nan = diff[~np.isnan(diff)]
            diff_delta_tests[i] = diff_delta_is_not_nan[-1]
            diff_sigma_tests[i] = diff_sigma_is_not_nan[-1]
            diff_tests[i] = diff_is_not_nan[-1]
    diff_delta_results[name] = diff_delta_tests
    diff_sigma_results[name] = diff_sigma_tests
    diff_results[name] = diff_tests
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff delta = {diff_delta_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff sigma = {diff_sigma_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff = {diff_tests}")

Model = VB, dimensionality = 400, diff delta = [917.82025146 909.840271   951.82830811]
Model = VB, dimensionality = 400, diff sigma = [0.00473717 0.00540889 0.00574677]
Model = VB, dimensionality = 400, diff = [917.82501221 909.84570312 951.83404541]


In [46]:
for name in model_names:
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff delta = {diff_delta_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff sigma = {diff_sigma_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff = {diff_results[name].mean()}")

Model = VB, dimensionality = 400, mean diff delta = 926.4962768554688
Model = VB, dimensionality = 400, mean diff sigma = 0.005297609294454257
Model = VB, dimensionality = 400, mean diff = 926.5015869140625


In [8]:
def run_test(d, params, 
             batch_size=256,
             clip_value=None,
             dtype=torch.float32, 
             device=torch.device("cpu")):
    """ 
    Run the gaussian test with dimension d 
    """

    ######### Problem Specification

    # Data generation parameters
    prior_mu_z = torch.zeros(d, dtype=dtype)    # Prior mean
    prior_sigma_z = torch.eye(d, dtype=dtype)   # Prior covariance matrix

    # True model parameters
    num_range = torch.arange(-(d-1)/2, (d+1)/2, dtype=dtype)

    t_delta =  num_range / 5 

    print(f"true delta = {t_delta.numpy()}")

    if d == 1:
        t_sigma = torch.ones(1, dtype=dtype)
    else: 
        # Allow sigma to range from 0.1 to 1
        t_sigma = 36/(10*(d-1)**2) * num_range**2 + 0.1 
    print(f"true sigma = {t_sigma.numpy()}")

    ######### Variable Initialization

    # Initial model parameters - same across all methods
    init_delta = prior_mu_z.clone().to(device)
    init_log_sigma = 3 * torch.ones(d).to(device)

    # Initial HVAE variational parameters
    init_T = torch.tensor(5., dtype=dtype, device=device)
    init_eps = 0.005 * torch.ones(d, dtype=dtype)
    max_eps = params['max_eps'] * torch.ones(d, dtype=dtype)
    init_logit_eps = torch.log(init_eps/(max_eps - init_eps)).to(device)
    init_log_T_0 = torch.log(init_T - 1)

    # Initial NF variational parameters
    init_u_pre_reparam = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, 
                                                                scale=0.1, 
                                                                size=d),
                                      dtype=dtype, device=device)
    init_w = torch.tensor(scipy.stats.truncnorm.rvs(-2, 2, scale=0.1, size=d),
                          dtype=dtype, device=device)
    init_b = torch.tensor(0.1, dtype=dtype, device=device)

    # Initial VAE parameters
    init_mu_z = prior_mu_z.clone()
    init_log_sigma_z = torch.ones(d, dtype=dtype)

    ######### Set up models
    init_params = [[init_delta, init_log_sigma],
                   [init_delta, init_log_sigma]]
    
    hid_dim = d + 100
    IAF_model = IAF(params,
        ['delta', 'log_sigma'],
        init_params[0], 
        'IAF', d, 1, hid_dim)

    #model_list = [HVAE_model_1, HVAE_model_2, NF_model_1]
    #model_list = [HVAE_model_1, NF_model_1]
    model_list = [IAF_model]

    print("Models were initialized!")
    
    ######### Generate Training Data & Save - One for each test

    train_data_list = []

    for i in range(params['n_tests']):
        z = MultivariateNormal(prior_mu_z, prior_sigma_z).sample()
        x = MultivariateNormal(z + t_delta, 
                               torch.diag(t_sigma**2)).sample([params['n_data']])
        x = x.to(device)
        train_data_list.append(x)

    # Folder should have already been created in the initializations
    data_path = os.path.join('save', str(d), 'train_data.p')
    pickle.dump(train_data_list, open(data_path, 'wb')) 

    # Store the final parameter values for all test runs in this dictionary
    final_params = {}

    for i in range(len(model_list)):
        final_values = []
        m = model_list[i]
        model_init_params = init_params[i]

        for j in range(params['n_tests']):
            print("------------------------------------------------------")
            print(f"Test = {j}")
            print("------------------------------------------------------")
            m._reinitialize(model_init_params)
            gaussian_data = gaussian_dataset(train_data_list[j])
            train_dataloader = DataLoader(gaussian_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
            (delta, sigma) = m.train(train_dataloader, j, 
                                     t_delta, t_sigma,
                                     clip_value)
            final_values.append((delta, sigma))

        final_params[m.model_name] = final_values.copy()

    ######### Test models using difference between parameters

    param_diffs = {}

    for m in model_list:
        
        diffs = []

        for i in range(params['n_tests']):
            delta = final_params[m.model_name][i][0]
            sigma = final_params[m.model_name][i][1]

            delta_diff = np.sum((delta - t_delta.numpy())**2)
            sigma_diff = np.sum((sigma - t_sigma.numpy())**2)
            theta_diff = delta_diff + sigma_diff
            print(f"Model {m.model_name}, test = {i}, result delta diff = {delta_diff}")
            print(f"Model {m.model_name}, test = {i}, result sigma diff = {sigma_diff}")
            print(f"Model {m.model_name}, test = {i}, result theta diff = {theta_diff}")

            diffs.append((delta_diff, sigma_diff, theta_diff))

        param_diffs[m.model_name] = diffs.copy()

    # Save parameter differences in a pickle file
    diff_path = os.path.join('save', str(d), 'all_diffs.p')
    pickle.dump(param_diffs, open(diff_path, 'wb'))

In [9]:
device = torch.device("cpu")
dtype = torch.float32
clip_value = None
batch_size = 256

In [10]:
params = {

    # PROBLEM SPECIFICATION
    #'dims': [1, 2, 3, 5, 11, 25, 51, 101, 201, 301], # Dimensions to test
    'dims': [25],
    'n_data': 10000,                                 # Number of data points

    # TEST HYPERPARAMETERS
    'n_tests': 3,      # Number of experiments to run

    # GLOBAL OPTIMIZATION PARAMETERS
    'n_iter': 2000,    # Number of optimization iterations
    'n_batch': 10,      # Number of points for ELBO estimate
    'rms_eta': 0.001,   # Stepsize for RMSProp
    'save_every': 20,   # Save parameter information every so often
    'print_every': 20, # Print less often than save

    # HVAE HYPERPARAMETERS
    'HVAE_K_1': 1,    # Number of leapfrog/cooling steps for HVAE flow 1
    'HVAE_K_2': 10,   # Number of leapfrog/cooling steps for HVAE flow 2
    'max_eps': 0.5,   # Maximum leapfrog step size per dimension 

    # NF HYPERPARAMETERS
    'NF_K_1': 1,    # Number of flow steps for NF flow 1
    'NF_K_2': 30,   # Number of flow steps for NF flow 2
    
}

In [11]:
dims = params['dims']
seed = 12345
np.random.seed(seed)
torch.manual_seed(seed)
for d in dims:
    print('**** Running test for d={0:d} ****'.format(d))
    run_test(d, params, batch_size, clip_value, dtype, device)

**** Running test for d=25 ****
true delta = [-2.4 -2.2 -2.  -1.8 -1.6 -1.4 -1.2 -1.  -0.8 -0.6 -0.4 -0.2  0.   0.2
  0.4  0.6  0.8  1.   1.2  1.4  1.6  1.8  2.   2.2  2.4]
true sigma = [1.         0.85625005 0.725      0.60625005 0.5        0.40625
 0.32500002 0.25625    0.2        0.15625    0.125      0.10625
 0.1        0.10625    0.125      0.15625    0.2        0.25625
 0.32500002 0.40625    0.5        0.60625005 0.725      0.85625005
 1.        ]
Models were initialized!
------------------------------------------------------
Test = 0
------------------------------------------------------
Init diff delta = 52.0
Init diff sigma = 9665.5205078125
Init diff theta = 9717.5205078125
IAF, d: 25, Iter: 1-20, s/iter: 8.105e-02, ELBO: -7.605e+05
Cur diff delta = 26.17352867126465
Cur diff sigma = 1461.0028076171875
Cur diff theta = 1487.1763362884521
---------------------------------------
IAF, d: 25, Iter: 1-40, s/iter: 8.028e-02, ELBO: -5.676e+05
Cur diff delta = 22.280956268310547
Cur 

In [12]:
model_names = ['IAF']
diff_delta_results = {}
diff_sigma_results = {}
diff_results = {}
for name in model_names:
    diff_delta_tests = np.zeros(params['n_tests'])
    diff_sigma_tests = np.zeros(params['n_tests'])
    diff_tests = np.zeros(params['n_tests'])
    for i in range(params['n_tests']):
        file_name = name + '_train_' + str(i) + '.p'
        diff_path = os.path.join('save', str(params['dims'][0]), file_name)
        with open(diff_path, 'rb') as f:
            data = pickle.load(f)
            elbo = data['elbo']
            diff_delta = data['diff_delta']
            diff_sigma = data['diff_sigma']
            diff = diff_delta + diff_sigma
            diff_delta_is_not_nan = diff_delta[~np.isnan(diff_delta)]
            diff_sigma_is_not_nan = diff_sigma[~np.isnan(diff_sigma)]
            diff_is_not_nan = diff[~np.isnan(diff)]
            diff_delta_tests[i] = diff_delta_is_not_nan[-1]
            diff_sigma_tests[i] = diff_sigma_is_not_nan[-1]
            diff_tests[i] = diff_is_not_nan[-1]
    diff_delta_results[name] = diff_delta_tests
    diff_sigma_results[name] = diff_sigma_tests
    diff_results[name] = diff_tests
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff delta = {diff_delta_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff sigma = {diff_sigma_tests}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, diff = {diff_tests}")

Model = IAF, dimensionality = 25, diff delta = [25.88096809 20.54680443 19.69295883]
Model = IAF, dimensionality = 25, diff sigma = [1.13933897 1.12645912 1.15877843]
Model = IAF, dimensionality = 25, diff = [27.02030754 21.67326355 20.85173798]


In [13]:
for name in model_names:
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff delta = {diff_delta_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff sigma = {diff_sigma_results[name].mean()}")
    print(f"Model = {name}, dimensionality = {params['dims'][0]}, mean diff = {diff_results[name].mean()}")

Model = IAF, dimensionality = 25, mean diff delta = 22.04024378458659
Model = IAF, dimensionality = 25, mean diff sigma = 1.1415255069732666
Model = IAF, dimensionality = 25, mean diff = 23.181769688924152
